In [ ]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Extract Products Data from Dataset

This step is completed using Talend Open Studio

# Data Sampling

the dataset is sampled into 10% of its original size is due to the hardware limitation to handle the large amount of sata entries. Additionally, sampling helps the model to be more generalised and towards unseen data.

In [ ]:
file_path_prefix = "/content/drive/MyDrive/datasets/"

file1_name = "cleaned-2019-Oct.csv"
file2_name = "cleaned-2019-Nov.csv"
file3_name = "cleaned-2019-Dec.csv"
file4_name = "cleaned-2020-Jan.csv"
file5_name = "cleaned-2020-Feb.csv"
file6_name = "cleaned-2020-Mar.csv"
file7_name = "cleaned-2020-Apr.csv"

filename_list = [file1_name, file2_name, file3_name, file4_name, file5_name, file6_name, file7_name]

product_dataset_filename = "product_dataset.csv"

In [ ]:
random_seed = 0
sampling_percentage = 0.1
chunk_size = 1000000

for filename in filename_list:

  # read the dataset files in chunks
  chunks = pd.read_csv(file_path_prefix + filename, chunksize=chunk_size)
  print(f"reading {filename}")
  output_file = file_path_prefix+"sampled-data/sampled-"+filename

  for index, chunk in enumerate(chunks):

    with open(output_file, 'a', newline='') as output_csv:

      sampled_df = chunk.sample(
          frac=sampling_percentage,
          random_state=random_seed
          )
      # append the filtered data into the output csv file
      if index == 0: sampled_df.to_csv(output_csv, index=False, header=True)
      else: sampled_df.to_csv(output_csv, index=False, header=False)
      # print(filtered_chunk.head(10))

  print(f"{filename} completed")

reading cleaned-2020-Jan.csv
cleaned-2020-Jan.csv completed


# Preprocess Product Data

All the information of each unique product are extracted from the ecommerce user behavior dataset to create a product information dataset.
This step ensures that no different products can share the same product id to prevent data inconsistentcy

In [ ]:
file_path_prefix = "/content/drive/MyDrive/datasets/sampled-data/"

file1_name = "sampled-cleaned-2019-Oct.csv"
file2_name = "sampled-cleaned-2019-Nov.csv"
file3_name = "sampled-cleaned-2019-Dec.csv"
file4_name = "sampled-cleaned-2020-Jan.csv"
file5_name = "sampled-cleaned-2020-Feb.csv"
file6_name = "sampled-cleaned-2020-Mar.csv"
file7_name = "sampled-cleaned-2020-Apr.csv"

filename_list = [file1_name, file2_name, file3_name, file4_name, file5_name, file6_name, file7_name]

product_dataset_filename = "sampled-product-dataset.csv"

In [ ]:
product_dataset_set = set()
product_df = pd.read_csv(file_path_prefix + product_dataset_filename, usecols=['product_id', 'category_id'])
product_dataset_set.update(set(zip(product_df['product_id'], product_df['category_id'])))

In [ ]:
product_dataset_set

In [ ]:
chunk_size = 1000000

event_weight = {
    'view': 1,
    'cart': 2,
    'remove_from_cart': -1,
    'purchase': 3,
}

for filename in filename_list:

  # read the dataset files in chunks
  chunks = pd.read_csv(file_path_prefix + filename, chunksize=chunk_size)
  print(f"reading {filename}")
  output_file = file_path_prefix+"filtered-"+filename

  for index, chunk in enumerate(chunks):

    with open(output_file, 'a', newline='') as output_csv:

      # filter out the rows that do not exist in the product table
      filtered_chunk = chunk[(
          chunk[['product_id', 'category_id']]
          .apply(tuple, axis=1)
          .isin(product_dataset_set)
      )]

      # add a rating column for each event
      filtered_chunk = filtered_chunk.assign(
          rating=filtered_chunk['event_type'].map(event_weight).fillna(0)
          )

      # append the filtered data into the output csv file
      if index == 0:
        filtered_chunk.to_csv(output_csv, index=False, header=True)
      else:
        filtered_chunk.to_csv(output_csv, index=False, header=False)
      # print(filtered_chunk.head(10))

  print(f"{filename} completed")

reading sampled-cleaned-2019-Oct.csv
sampled-cleaned-2019-Oct.csv completed
reading sampled-cleaned-2019-Nov.csv
sampled-cleaned-2019-Nov.csv completed
reading sampled-cleaned-2019-Dec.csv
sampled-cleaned-2019-Dec.csv completed
reading sampled-cleaned-2020-Jan.csv
sampled-cleaned-2020-Jan.csv completed
reading sampled-cleaned-2020-Feb.csv
sampled-cleaned-2020-Feb.csv completed
reading sampled-cleaned-2020-Mar.csv
sampled-cleaned-2020-Mar.csv completed
reading sampled-cleaned-2020-Apr.csv
sampled-cleaned-2020-Apr.csv completed


In [ ]:
chunk_size = 1000000

event_weight = {
    'view': 1,
    'cart': 2,
    'remove_from_cart': -1,
    'purchase': 3,
}

for filename in filename_list:
  chunks = pd.read_csv(file_path_prefix + filename, chunksize=chunk_size)
  print(f"reading {filename}")
  output_file = file_path_prefix+"filtered-"+filename
  for index, chunk in enumerate(chunks):
    with open(output_file, 'a', newline='') as output_csv:
      filtered_chunk = chunk[(
          chunk[['product_id', 'category_id']]
          .apply(tuple, axis=1)
          .isin(product_dataset_set)
      )]
      filtered_chunk = filtered_chunk.assign(
          rating=filtered_chunk['event_type'].map(event_weight).fillna(0)
          )
      if index == 0: filtered_chunk.to_csv(output_csv, index=False, header=True)
      else: filtered_chunk.to_csv(output_csv, index=False, header=False)

  print(f"{filename} completed")

In [ ]:
filename = "/content/drive/MyDrive/datasets/sampled-data/filtered-sampled-cleaned-2019-Dec.csv"
temp = pd.read_csv(filename, nrows=100)
temp

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session,rating
0,2019-12-01 06:30:35 UTC,view,10504428,2053013553098522884,kids.toys,lolsurprise,77.20,553306173,a846c1e8-4518-4c2a-a250-7b0de739e971,1
1,2019-12-01 07:22:14 UTC,view,1306925,2053013558920217191,computers.notebook,hp,1171.18,574736030,dbe39870-5396-4014-be2b-8fed92667f2d,1
2,2019-12-01 13:08:09 UTC,purchase,37900008,2232732107614978731,apparel.shorts,merelani,129.99,533331488,cae838cb-ca7e-4835-a197-293f51141b16,3
3,2019-12-01 07:58:02 UTC,view,14300017,2053013557603205653,electronics.audio.music_tools.piano,cortland,90.07,527900162,ac2f0e8b-6744-4f39-9ae9-a8a81d9c7a0a,1
4,2019-12-01 10:13:38 UTC,view,4802983,2053013554658804075,electronics.audio.headphone,meizu,30.86,579492833,ec704a3d-da3d-44fc-b894-d6f1bee23c77,1
...,...,...,...,...,...,...,...,...,...,...
95,2019-12-01 11:01:05 UTC,view,3701366,2053013565983425517,appliances.environment.vacuum,ilife,254.83,513408339,f2978409-b039-4e77-8298-c69c9561f078,1
96,2019-12-01 15:30:12 UTC,view,37900146,2053013558752445019,auto.accessories.compressor,vega,41.70,541978811,a5293e55-0628-4b88-a17c-4eec7d1aa811,1
97,2019-12-01 13:46:35 UTC,view,17300968,2232732098446229999,apparel.shoes.sandals,afnan,55.34,535414839,7f9e7626-3a4b-4562-8c65-0e8053ae4525,1
98,2019-12-01 15:09:51 UTC,view,26400283,2053013553056579841,computers.peripherals.printer,lucente,232.44,513815977,d95d6910-c5fb-4e3b-a7b1-ce47c88b5ea4,1


# Extract User Purchases Data

The users' past purchasing histories are extracted from the dataset for recommendation product filtering purposes.

In [ ]:
file_path_prefix = "/content/drive/MyDrive/datasets/sampled-data/"

file1_name = "filtered-sampled-cleaned-2019-Oct.csv"
file2_name = "filtered-sampled-cleaned-2019-Nov.csv"
file3_name = "filtered-sampled-cleaned-2019-Dec.csv"
file4_name = "filtered-sampled-cleaned-2020-Jan.csv"
file5_name = "filtered-sampled-cleaned-2020-Feb.csv"
file6_name = "filtered-sampled-cleaned-2020-Mar.csv"
file7_name = "filtered-sampled-cleaned-2020-Apr.csv"

filename_list = [file1_name, file2_name, file3_name, file4_name, file5_name, file6_name, file7_name]

In [ ]:
chunk_size = 1000000
output_file = file_path_prefix+"purchase_history.csv"
total_row_extracted = 0
with open(output_file, 'a', newline='') as output_csv:
  for filename in filename_list:
    chunks = pd.read_csv(file_path_prefix + filename, chunksize=chunk_size)
    print(f"reading {filename} ...")

    for index, chunk in enumerate(chunks):
      purchase_chunk = chunk[chunk['event_type'] == 'purchase']
       [['event_time', 'product_id', 'user_id']]

      if total_row_extracted == 0:
        purchase_chunk.to_csv(output_csv, index=False, header=True)
      else:
        purchase_chunk.to_csv(output_csv, index=False, header=False)

      total_row_extracted += purchase_chunk.size

    print(f"total row extracted: {total_row_extracted}")
    print(f"-> {filename} completed")

reading filtered-sampled-cleaned-2019-Oct.csv ...
total row extracted: 164583
-> filtered-sampled-cleaned-2019-Oct.csv completed
reading filtered-sampled-cleaned-2019-Nov.csv ...
total row extracted: 360918
-> filtered-sampled-cleaned-2019-Nov.csv completed
reading filtered-sampled-cleaned-2019-Dec.csv ...
total row extracted: 415179
-> filtered-sampled-cleaned-2019-Dec.csv completed
reading filtered-sampled-cleaned-2020-Jan.csv ...
total row extracted: 453981
-> filtered-sampled-cleaned-2020-Jan.csv completed
reading filtered-sampled-cleaned-2020-Feb.csv ...
total row extracted: 518835
-> filtered-sampled-cleaned-2020-Feb.csv completed
reading filtered-sampled-cleaned-2020-Mar.csv ...
total row extracted: 586566
-> filtered-sampled-cleaned-2020-Mar.csv completed
reading filtered-sampled-cleaned-2020-Apr.csv ...
total row extracted: 659238
-> filtered-sampled-cleaned-2020-Apr.csv completed
